In [1]:
import pandas as pd

### Team level statistics to consider
- Average score for each team
- Count of each tag for each team 
- 'Efficiency rating' that compares number of tags to word count (i.e.  1000 words with 10 tags is rated lower than 100 words with 10 tags)

In [37]:
df = pd.read_csv('virtualInternshipData_ADS2001.csv', encoding = 'unicode_escape')

df.head()

,Unnamed: 0,userIDs,implementation,Line_ID,ChatGroup,content,group_id,RoleName,roomName,m_experimental_testing,m_making_design_choices,m_asking_questions,j_customer_consultants_requests,j_performance_parameters_requirements,j_communication,OutcomeScore,wordCount
0,1,1,a,1,PRNLT,Hello team. Welcome to Nephrotex!,2,Mentor,Introduction and Workflow Tutorial with Entran...,0,0,0,0,0,0,4,5
1,2,1,a,2,PRNLT,I'm Maria Williams. I'll be your design adviso...,2,Mentor,Introduction and Workflow Tutorial with Entran...,0,0,0,0,0,0,4,11
2,3,1,a,3,PRNLT,I'm here to help if you have any questions.,2,Mentor,Introduction and Workflow Tutorial with Entran...,0,0,0,0,0,0,4,9
3,4,1,a,4,PRNLT,Please introduce yourselves with the name you ...,2,Mentor,Introduction and Workflow Tutorial with Entran...,0,0,0,1,0,0,4,51
4,5,1,a,5,PRNLT,I just want to make sure everyone has found th...,2,Mentor,Introduction and Workflow Tutorial with Entran...,0,0,0,0,0,0,4,39


Find the different groups in the dataset

In [27]:
df['group_id'].unique()

array([2, 3, 4, 5, 6], dtype=int64)

Split groups into their own seperate dataframes

In [43]:
group2 = df[df['group_id'] == 2]
group3 = df[df['group_id'] == 3]
group4 = df[df['group_id'] == 4]
group5 = df[df['group_id'] == 5]
group6 = df[df['group_id'] == 6]

group2.head()

,Unnamed: 0,userIDs,implementation,Line_ID,ChatGroup,content,group_id,RoleName,roomName,m_experimental_testing,m_making_design_choices,m_asking_questions,j_customer_consultants_requests,j_performance_parameters_requirements,j_communication,OutcomeScore,wordCount
0,1,1,a,1,PRNLT,Hello team. Welcome to Nephrotex!,2,Mentor,Introduction and Workflow Tutorial with Entran...,0,0,0,0,0,0,4,5
1,2,1,a,2,PRNLT,I'm Maria Williams. I'll be your design adviso...,2,Mentor,Introduction and Workflow Tutorial with Entran...,0,0,0,0,0,0,4,11
2,3,1,a,3,PRNLT,I'm here to help if you have any questions.,2,Mentor,Introduction and Workflow Tutorial with Entran...,0,0,0,0,0,0,4,9
3,4,1,a,4,PRNLT,Please introduce yourselves with the name you ...,2,Mentor,Introduction and Workflow Tutorial with Entran...,0,0,0,1,0,0,4,51
4,5,1,a,5,PRNLT,I just want to make sure everyone has found th...,2,Mentor,Introduction and Workflow Tutorial with Entran...,0,0,0,0,0,0,4,39


In [5]:
print("The average score for group 2 is", round(group2['OutcomeScore'].mean(),2))
print("The average score for group 3 is", round(group3['OutcomeScore'].mean(),2))
print("The average score for group 4 is", round(group4['OutcomeScore'].mean(),2))
print("The average score for group 5 is", round(group5['OutcomeScore'].mean(),2))
print("The average score for group 6 is", round(group6['OutcomeScore'].mean(),2))

The average score for group 2 is 3.47
The average score for group 3 is 3.71
The average score for group 4 is 3.8
The average score for group 5 is 4.08
The average score for group 6 is 3.68


In [93]:
tagCount2 = group2.groupby('group_id',axis=0).sum()
tagCount2 = tagCount2.iloc[:,3:9]

tagCount3 = group3.groupby('group_id',axis=0).sum()
tagCount3 = tagCount3.iloc[:,3:9]

tagCount4 = group4.groupby('group_id',axis=0).sum()
tagCount4 = tagCount4.iloc[:,3:9]

tagCount5 = group5.groupby('group_id',axis=0).sum()
tagCount5 = tagCount5.iloc[:,3:9]

tagCount6 = group6.groupby('group_id',axis=0).sum()
tagCount6 = tagCount6.iloc[:,3:9]

tagdf = tagCount2

tagdf = tagdf.append(tagCount3)
tagdf = tagdf.append(tagCount4)
tagdf = tagdf.append(tagCount5)
tagdf = tagdf.append(tagCount6)

tagdf.head()

,m_experimental_testing,m_making_design_choices,m_asking_questions,j_customer_consultants_requests,j_performance_parameters_requirements,j_communication
group_id,,,,,,
2,109,401,743,71,220,86
3,67,382,729,95,190,84
4,161,415,783,64,209,83
5,117,407,660,64,183,74
6,97,368,672,54,200,77
